# 基于深度学习创建自定义表情符号项目详解

## 一、项目背景
在当今数字化交流日益频繁的时代，表情符号（Emojis）已成为在线聊天、产品评论、品牌情感表达等众多场景中不可或缺的一部分。它能够传达非语言线索，丰富交流内容。随着计算机视觉和深度学习技术的飞速发展，从图像中检测人类情感并将其与表情符号建立联系成为了可能。本项目旨在利用深度学习技术，构建一个能够将人类面部表情分类，并映射到相应表情符号的系统。

## 二、数据集介绍
本项目使用的是FER2013（Facial Expression Recognition 2013）数据集。
1. **数据规格**：该数据集由48×48像素的灰度面部图像组成。这些图像均经过处理，使其居中且在图像中占据相等的空间。
2. **情感类别**：共包含7种面部情感类别，分别通过数字进行标识：
    - 0：愤怒（angry）
    - 1：厌恶（disgust）
    - 2：恐惧（fear）
    - 3：快乐（happy）
    - 4：悲伤（sad）
    - 5：惊讶（surprise）
    - 6：自然（natural）

可通过提供的链接下载该数据集：[Facial Expression Recognition Dataset](下载链接)

项目相关链接：
- https://www.kaggle.com/datasets/msambare/fer2013?resource=download

- https://data-flair.training/blogs/create-emoji-with-deep-learning/
## 三、项目实现步骤

### （一）面部表情识别（使用CNN）
1. **下载与准备数据集**
    - 从上述指定链接下载FER2013数据集。
    - 在本地创建一个名为`data`的文件夹，将下载后的数据集解压到该文件夹中，并确保在`data`文件夹下分别创建`train`和`test`两个子目录，用于存放训练数据和测试数据。

2. **创建`train.py`文件并编写代码**
    - **导入必要的库**


In [5]:
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator


- `numpy`：用于处理数值计算和数组操作。
- `cv2`：OpenCV库，用于计算机视觉任务，如读取图像、图像处理等。
- `Sequential`：Keras中的序贯模型，用于按顺序堆叠神经网络层。
- `Dense`：全连接层，用于神经网络中的计算。
- `Dropout`：用于防止过拟合，在训练过程中随机丢弃一些神经元。
- `Flatten`：将多维数据展平为一维数据，以便输入到全连接层。
- `Conv2D`：二维卷积层，用于提取图像特征。
- `Adam`：一种优化器，用于更新神经网络的权重。
- `MaxPooling2D`：二维最大池化层，用于下采样，减少数据量和计算量。
- `ImageDataGenerator`：用于数据增强和生成训练数据和验证数据的生成器。


- **初始化训练和验证数据生成器**

In [9]:
# 初始化数据生成器
train_dir = 'C:/Users/admin/Desktop/project/Emoji/archive/train'
val_dir = 'C:/Users/admin/Desktop/project/Emoji/archive/test'
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)
validation_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(48, 48),
    batch_size=64,
    color_mode="grayscale",
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


- `train_dir`和`val_dir`分别指定训练数据和验证数据的目录。
- `train_datagen`和`val_datagen`是`ImageDataGenerator`的实例，`rescale=1./255`表示将图像像素值归一化到0 - 1之间，因为原始图像像素值范围是0 - 255。
- `train_generator`和`validation_generator`通过`flow_from_directory`方法从指定目录中读取图像数据，`target_size=(48,48)`将图像调整为48×48像素大小，`batch_size=64`表示每次从数据集中读取64个样本作为一个批次进行训练，`color_mode="grayscale"`指定图像为灰度图，`class_mode='categorical'`表示标签采用独热编码（one - hot encoding）形式。

- **构建卷积神经网络架构**

In [14]:
# 构建模型
emotion_model = Sequential()
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

E:\Anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


- 模型的第一层是`Conv2D`层，有32个卷积核，卷积核大小为(3, 3)，激活函数使用`relu`，`input_shape=(48,48,1)`表示输入图像的形状为48×48像素的单通道灰度图。
- 接着是多个`Conv2D`层和`MaxPooling2D`层的组合。`Conv2D`层用于提取图像的特征，随着层数的增加，卷积核的数量逐渐增多（如从32到64再到128），以提取更复杂的特征。`MaxPooling2D`层用于下采样，通过池化操作（这里池化大小为(2, 2)），将特征图的尺寸减半，减少计算量和参数数量，同时保留主要特征。
- `Dropout`层用于防止过拟合，在训练过程中随机丢弃一定比例（如0.25或0.5）的神经元，使模型学习到更鲁棒的特征。
- `Flatten`层将多维的特征图展平为一维向量，以便输入到全连接层。
- 全连接层`Dense`，第一个`Dense`层有1024个神经元，激活函数为`relu`；最后一个`Dense`层有7个神经元，对应7种情感类别，激活函数为`softmax`，用于输出每个类别的概率。

- **编译和训练模型**

In [27]:
# 训练模型，使用 fit 方法
emotion_model_info = emotion_model.fit(
    train_generator,
    steps_per_epoch=28709 // 64,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=7178 // 64
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/50


E:\Anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


448/448 ━━━━━━━━━━━━━━━━━━━━ 100s 219ms/step - accuracy: 0.2458 - loss: 1.8322 - val_accuracy: 0.3188 - val_loss: 1.7348
Epoch 2/50
  1/448 ━━━━━━━━━━━━━━━━━━━━ 1:38 220ms/step - accuracy: 0.2344 - loss: 1.8257

E:\Anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:107: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


448/448 ━━━━━━━━━━━━━━━━━━━━ 7s 16ms/step - accuracy: 0.2344 - loss: 1.8257 - val_accuracy: 0.3237 - val_loss: 1.7365
Epoch 3/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 98s 220ms/step - accuracy: 0.3324 - loss: 1.6857 - val_accuracy: 0.4171 - val_loss: 1.5513
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.3906 - loss: 1.4768 - val_accuracy: 0.4156 - val_loss: 1.5511
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 109s 244ms/step - accuracy: 0.4057 - loss: 1.5494 - val_accuracy: 0.4431 - val_loss: 1.4762
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 7s 14ms/step - accuracy: 0.4062 - loss: 1.5000 - val_accuracy: 0.4460 - val_loss: 1.4744
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 99s 221ms/step - accuracy: 0.4309 - loss: 1.4787 - val_accuracy: 0.4708 - val_loss: 1.3979
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.4688 - loss: 1.4014 - val_accuracy: 0.4746 - val_loss: 1.4013
Epoch 9/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 89s 199ms/step - accuracy: 0.4666 - loss: 1.4029 - val_accura

- `emotion_model.compile`用于编译模型，`loss='categorical_crossentropy'`表示使用分类交叉熵损失函数，适用于多分类问题；`optimizer=Adam(lr=0.0001, decay=1e-6)`使用Adam优化器，设置初始学习率为0.0001，学习率衰减为`1e - 6`；`metrics=['accuracy']`表示在训练过程中监控模型的准确率。
- `emotion_model.fit_generator`用于训练模型，`train_generator`为训练数据生成器，`steps_per_epoch=28709 // 64`表示每个epoch需要训练的步数，`28709`是训练数据集中样本的总数，除以`batch_size=64`得到步数；`epochs=50`表示训练50个epoch；`validation_data=validation_generator`指定验证数据生成器，`validation_steps=7178 // 64`表示每个epoch验证时的步数，`7178`是验证数据集中样本的总数。

- **保存模型权重**

In [31]:
emotion_model.save_weights('model.weights.h5')

将训练好的模型权重保存为`model.h5`文件，以便后续使用。

- **使用OpenCV进行人脸检测和情感预测**

In [7]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # 使用 cv2.data.haarcascades 动态获取 Haar 级联分类器文件的路径
    bounding_box = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y - 50), (x + w, y + h + 10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        cv2.putText(frame, emotion_dict[maxindex], (x + 20, y - 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame, (1200, 860), interpolation=cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break

- `cv2.ocl.setUseOpenCL(False)`：禁用OpenCL加速，因为在某些情况下可能会导致问题。
- `emotion_dict`是一个字典，用于将模型预测的数字标签映射到对应的情感类别字符串。
- `cap = cv2.VideoCapture(0)`：打开默认的摄像头设备。
- 在循环中，通过`cap.read()`读取摄像头的每一帧图像。如果读取失败（`not ret`），则退出循环。
- 使用OpenCV的Haar级联分类器`cv2.CascadeClassifier`加载人脸检测模型（`haarcascade_frontalface_default.xml`），将彩色图像转换为灰度图像`gray_frame`，然后使用`detectMultiScale`方法检测灰度图像中的人脸，`scaleFactor=1.3`表示每次图像缩放的比例，`minNeighbors=5`表示每个候选矩形需要保留的邻居数，以确定是否为真正的人脸。
- 对于检测到的每一个人脸，使用`cv2.rectangle`在原始图像上绘制矩形框标记人脸。然后提取人脸区域`roi_gray_frame`，对其进行缩放和维度扩展，使其符合模型输入的要求（形状为(1, 48, 48, 1)）。
- 使用训练好的模型`emotion_model.predict`对提取的人脸图像进行情感预测，得到一个概率数组。通过`np.argmax`找到概率最大的索引，即预测的情感类别索引`maxindex`。
- 使用`cv2.putText`在图像上显示预测的情感类别字符串。
- 使用`cv2.imshow`显示处理后的图像，`cv2.waitKey(1)`等待按键事件，如果按下`q`键，则释放摄像头资源并关闭所有OpenCV窗口。


## 四、项目总结
本项目通过构建卷积神经网络（CNN），在FER2013数据集上进行训练，实现了面部表情的识别，并将识别出的情感映射到相应的表情符号。利用OpenCV的Haar级联分类器进行人脸检测，同时通过`tkinter`创建了图形用户界面，方便用户直观地看到面部表情对应的表情符号。通过这个项目，初学者可以深入了解深度学习在计算机视觉领域的应用，包括数据集处理、模型构建、训练以及GUI开发等多个方面。 